# Análise das despesas da prefeitura de João Pessoa

#### [Fonte: Dados abertos do Portal da transparência da Prefeitura de João Pessoa](https://transparencia.joaopessoa.pb.gov.br/#/dados-abertos/listagem/savings/Receitas%20e%20Despesas)

In [11]:
#Imports
%pip install pandas
%pip install scikit-learn
import pandas as pd
from sklearn.cluster import KMeans

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Read data
pd.set_option("float_format",'{:.2f}'.format)
df = pd.read_csv(filepath_or_buffer='./data/2022.csv', sep=';', low_memory=False)
df = df.drop(["sald_ante","valo_orca","tota_cred","nume_proc_lici","nume_proc","nume_lici"], axis=1)
df = df[df.columns[:-1]]
df.head(3)

,ano_refe,mes_refe,codi_enti,nome_enti,desc_forc,nume_empe,ano_empe,data_empe,desc_tpem,nome_forn,...,codi_unor,desc_unor,codi_func,codi_subf,codi_prog,codi_acao,desc_acao,codi_sube,desc_sube,ug
0,2022,2,13,Fundo Municipal de Saude de Joao Pessoa,#Transferencias SUS do Governo Federal - Manut...,422,2022,2022-02-09 00:00:00,Global,CENTRAL DE CARGAS ASA BRANCA,...,130301,FUNDO MUNICIPAL DE SAUDE,10,302,5005,386,MAC - REDE HOSPITALAR - MANTER E IMPLEMENTAR O...,33903921,LOCACAO DE IMOVEIS,610095
1,2022,7,13,Fundo Municipal de Saude de Joao Pessoa,#Transferencias SUS do Governo Federal - Manut...,2632,2022,2022-07-18 00:00:00,Global,SISPACK MEDICAL LTDA,...,130301,FUNDO MUNICIPAL DE SAUDE,10,302,5005,386,MAC - REDE HOSPITALAR - MANTER E IMPLEMENTAR O...,33903016,MATERIAL BIOLOGICO,610095
2,2022,7,13,Fundo Municipal de Saude de Joao Pessoa,#Transferencias SUS do Governo Federal - Manut...,2631,2022,2022-07-18 00:00:00,Global,SISPACK MEDICAL LTDA,...,130301,FUNDO MUNICIPAL DE SAUDE,10,302,5005,387,MAC - ACOES DE MEDIA E ALTA COMPLEXIDADE - MAN...,33903016,MATERIAL BIOLOGICO,610095


In [13]:
# Conversão de valores
df["data_empe"] = pd.to_datetime(df["data_empe"])
df["trimestre_refe"] = df["data_empe"].dt.quarter
df.dtypes

ano_refe                   int64
mes_refe                   int64
codi_enti                  int64
nome_enti                 object
desc_forc                 object
nume_empe                  int64
ano_empe                   int64
data_empe         datetime64[ns]
desc_tpem                 object
nome_forn                 object
codi_elem                  int64
desc_desp                 object
desc_orga                 object
desc_tpde                 object
desc_tpli                 object
valo_empe                float64
data_movi                 object
desc_tpmo                 object
nume_parc                  int64
valo_movi                float64
data_ulal                 object
desc_itee                 object
desc_func                 object
desc_subf                 object
desc_prog                 object
cnpj_cpf                  object
desc_proj                 object
codi_orga                  int64
codi_unor                  int64
desc_unor                 object
codi_func 

In [14]:
valores_unicos = pd.DataFrame(map(lambda columnn:
    {
        "coluna": columnn,
        "quantidade": df[columnn].nunique(),
        "valores": df[columnn].unique()
    } ,df.columns))
valores_unicos


,coluna,quantidade,valores
0,ano_refe,1,[2022]
1,mes_refe,12,"[2, 7, 3, 4, 5, 6, 8, 9, 10, 11, 1, 12]"
2,codi_enti,17,"[13, 1, 2, 10, 42, 23, 24, 14, 34, 18, 6, 33, ..."
3,nome_enti,17,"[Fundo Municipal de Saude de Joao Pessoa, Pref..."
4,desc_forc,25,[#Transferencias SUS do Governo Federal - Manu...
5,nume_empe,3750,"[422, 2632, 2631, 78, 55, 132, 194, 217, 250, ..."
6,ano_empe,1,[2022]
7,data_empe,258,"[2022-02-09 00:00:00, 2022-07-18 00:00:00, 202..."
8,desc_tpem,3,"[Global, Ordinario, Estimativo]"
9,nome_forn,726,"[CENTRAL DE CARGAS ASA BRANCA, SISPACK MEDICAL..."


In [15]:
total_despesa= df["valo_empe"].sum()
total_despesa

1702987884.6

### Participação por entidade

In [16]:
participacao_entidades = df[["codi_enti","nome_enti","valo_empe"]].groupby(["codi_enti","nome_enti"]).sum()
participacao_entidades["%"] =  (participacao_entidades["valo_empe"]/total_despesa)*100
participacao_entidades

,,valo_empe,%
codi_enti,nome_enti,,
1,Prefeitura Municipal de Joao Pessoa,1072524839.64,62.98
2,Autarquia Especial Municipal de Limpeza Urbana - EMLUR,87454838.21,5.14
6,Instituto de Previdencia do Municipio de Joao Pessoa,77117892.17,4.53
10,Fundacao Cultural de Joao Pessoa,5505730.60,0.32
11,Fundo Municipal de Cultura,21000.00,0.00
13,Fundo Municipal de Saude de Joao Pessoa,427149093.96,25.08
14,Fundo Municipal dos Dir. da Crianca e do Adolescente de Joao Pessoa,1369257.24,0.08
18,Fundo Municipal de Defesa dos Direitos Difusos,457266.54,0.03
23,Instituto Candida Vargas,9691742.85,0.57


### Participação por Tipo Orçamento

In [17]:
participacao_orcamento = df[["codi_unor","desc_unor","valo_empe"]].groupby(["codi_unor","desc_unor"]).sum()
participacao_orcamento["%"] =  (participacao_orcamento["valo_empe"]/total_despesa)*100
participacao_orcamento

,,valo_empe,%
codi_unor,desc_unor,,
30101,GABINETE DO VICE-PREFEITO,2894018.48,0.17
50101,GABINETE DO PROCURADOR,14040847.59,0.82
50301,"FUNDO DE GESTAO, DESENV. E MODERNIZACAO DA PROGEM",222045.55,0.01
60101,SECRETARIA DA ADMINISTRACAO-GABINETE DO SECRETARIO,13530271.38,0.79
60201,INSTITUTO DE PREVIDENCIA DO MUNICIPIO - IPM,77117892.17,4.53
70101,SECRETARIA DAS FINANCAS-GABINETE DO SECRETARIO,15672023.88,0.92
80101,SECRETARIA DE PLANEJAMENTO-GABINETE DO SECRETARIO,15328790.36,0.90
80102,UNIDADE EXECUTORA MUNICIPAL,2073687.76,0.12
80301,FUNDURB,1533.20,0.00


### Participação por função

In [18]:
participacao_funcao = df[["codi_func","desc_func","valo_empe"]].groupby(["codi_func","desc_func",]).sum()
participacao_funcao["%"] =  (participacao_funcao["valo_empe"]/total_despesa)*100
participacao_funcao

,,valo_empe,%
codi_func,desc_func,,
2,Judiciaria,77583.45,0.00
4,Administracao,264629864.72,15.54
6,Seguranca Publica,27418346.08,1.61
8,Assistencia Social,16767105.71,0.98
9,Previdencia Social,80673489.91,4.74
10,Saude,430006040.82,25.25
11,Trabalho,466382.78,0.03
12,Educacao,585323669.50,34.37
13,Cultura,6084545.77,0.36


In [19]:
participacao_trimestre = df[["trimestre_refe","valo_empe"]].groupby(["trimestre_refe",]).sum()
participacao_trimestre["%"] =  (participacao_trimestre["valo_empe"]/total_despesa)*100
participacao_trimestre

,valo_empe,%
trimestre_refe,,
1,502340640.66,29.50
2,316740067.01,18.60
3,425757665.24,25.00
4,458149511.69,26.90


In [20]:
df_filtro = df[["valo_empe","trimestre_refe", "codi_func","codi_unor","codi_enti"]]

kmeans = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(df_filtro)
labels = kmeans.labels_
labels


array([0, 0, 0, ..., 0, 0, 0])